In [1]:
import os
import cv2
import pandas as pd
import numpy as np
from skimage import feature

#for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#for data splitting
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#for the model prediction
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
data_dir     = './dataset anemia/'
india_folder = os.path.join(data_dir, 'India')
italy_folder = os.path.join(data_dir, 'Italy')


In [3]:
def should_use_img(img_name, img_type):
    if img_name.split(".")[1] in ["jpg", "png"]:
        img_name = img_name.split(".")[0]
        
        if img_type == "":
            img_types   = ["forniceal", "forniceal_palpebral", "palpebral"]
            for _type in img_types:
                if img_name.endswith(_type):
                    return False
            return True
        elif img_type == "palpebral":
            img_name = img_name.split(".")[0]
            if img_name.endswith("forniceal_palpebral"):
                    return False
            return True if img_name.endswith(img_type) else False
            
        else:
            return True if img_name.endswith(img_type)  else False
    
    else:
        return False
        

img_to_use  = "palpebral"
img_files   = {"India":[], "Italy":[]}

for folder in ['India', 'Italy']:
    img_folder = os.path.join(data_dir, folder)
    for root, dirs, files in os.walk(img_folder):
        flag = False
        for file in files:
            if should_use_img(file, img_to_use):
                cls = root.split("\\")[-1]
                img_files[folder].append( ( os.path.join(root, file), cls) )
                flag = True
                break
        if flag is False:
            print(root, " not found -> ", files)


./dataset anemia/India  not found ->  ['.DS_Store', 'IndiaRd.xlsx']
./dataset anemia/Italy  not found ->  ['.DS_Store', 'Italyrd.xlsx']


In [4]:
for key, items in img_files.items():
    print(f"items for {key} -> {len(items)}")

items for India -> 95
items for Italy -> 123


In [5]:
img_files["India"][0]

('./dataset anemia/India\\1\\20200118_164733_palpebral.png', '1')

In [6]:
data_dir = './dataset anemia/'
file1    = data_dir+'India/IndiaRd.xlsx'
file2    = data_dir+'Italy/Italyrd.xlsx'

In [7]:
labels = {}

d1 = pd.read_excel(file1)
d1["Anemia"] = d1["Note"]
d1 = d1[["Number", "Anemia" ]]
d1.replace("No anemia", "No Anemia", inplace=True)

d2 = pd.read_excel(file2)[["Number", "Anemia" ]]
d2.replace("No anemia", "No Anemia", inplace=True)

unique_values   = list(d1['Anemia'].unique())
enum            = {val:i for i, val in enumerate(unique_values)} 
print(enum)

d1['Anemia'].replace(enum, inplace=True)
d2['Anemia'].replace(enum, inplace=True)

labels["India"] = { row["Number"]:row["Anemia"] for index, row in d1.iterrows() }
labels["Italy"] = { row["Number"]:row["Anemia"] for index, row in d2.iterrows() }


{'No Anemia': 0, 'Anemia': 1}


## Load Images

### Feature Extractors

In [8]:
class LocalBinaryPatterns:
    def __init__(self, numPoints, radius):
        self.numPoints = numPoints
        self.radius = radius
    
    def preprocess(self, image):
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        return gray
    
    def describe(self, image, eps = 1e-7):
        image = self.preprocess(image)
        lbp   = feature.local_binary_pattern(image, self.numPoints, self.radius, method="uniform")
        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, self.numPoints+3), range=(0, self.numPoints + 2))

        # Normalize the histogram
        hist = hist.astype('float')
        hist /= (hist.sum() + eps)

        return hist, lbp

In [9]:
# LBP Hist

# Un-comment this code to generate LBPH features again OR use next tab to use pre-generaated features
"""
X = []
y = []

desc = LocalBinaryPatterns(24, 8)

for key, items in img_files.items():
    for item in items:
        img_path, cls = item
        img           = cv2.imread(img_path)
        hist, lbp     = desc.describe(img)
        lbl           = labels[key][int(cls)]
        
        X.append(hist)
        y.append(lbl)


import pickle

file = open("data_palpebral.pickle", "wb")
pickle.dump((X, y), file)
file.close()

"""

In [2]:
# Use this tab to load pre-generated features
import pickle

file = open("data_palpebral.pickle", "rb")
X, y = pickle.load(file)
file.close()

In [11]:
print(f"len(X) -> {len(X)}")
print(f"len(y) -> {len(y)}")

len(X) -> 218
len(y) -> 218


In [12]:
y = [0 if i == 0 else 1 for i in y]

# Training

In [13]:
X = np.array(X)
X.shape

(218, 26)

In [7]:
X[2]

array([0.02933798, 0.02768168, 0.00533081, 0.00256741, 0.00193696,
       0.00205869, 0.00253323, 0.00369055, 0.00558778, 0.0086782 ,
       0.013298  , 0.01938485, 0.0226868 , 0.02116112, 0.0143746 ,
       0.00940097, 0.00588331, 0.00411981, 0.00282455, 0.00248938,
       0.00245762, 0.00335008, 0.00690925, 0.03283306, 0.0376732 ,
       0.71175011])

In [14]:
Y = y.copy()
len(Y)

218

In [15]:
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, train_size=0.7, random_state=123)
print('Training Samples: '+str(len(X_train)) + ' \n Test Samples: ' + str(len(X_test)))

Training Samples: 152 
 Test Samples: 66


## Tf Model

In [18]:
import tensorflow as tf



In [16]:
X_train.shape

(152, 26)

In [19]:
y_train = tf.keras.utils.to_categorical(Y_train, 2)
y_test  = tf.keras.utils.to_categorical(Y_test, 2)

In [20]:
y_train.shape

(152, 2)

In [21]:
model =   tf.keras.models.Sequential([
                            tf.keras.layers.Dense(128, input_shape=(None, X_train.shape[1]), activation='relu'),
                            tf.keras.layers.Dense(56,  activation='relu'),
                            tf.keras.layers.Dense(2)
                            ])

In [22]:
loss_fn = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer='adam',loss=loss_fn, metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
5/5 [==============================] - 1s 2ms/step - loss: 9.7557 - accuracy: 0.3947
Epoch 2/50
5/5 [==============================] - 0s 2ms/step - loss: 9.7557 - accuracy: 0.3947
Epoch 3/50
5/5 [==============================] - 0s 3ms/step - loss: 9.7557 - accuracy: 0.3947
Epoch 4/50
5/5 [==============================] - 0s 3ms/step - loss: 9.7557 - accuracy: 0.3947
Epoch 5/50
5/5 [==============================] - 0s 2ms/step - loss: 9.7557 - accuracy: 0.3947
Epoch 6/50
5/5 [==============================] - 0s 3ms/step - loss: 9.7557 - accuracy: 0.3947
Epoch 7/50
5/5 [==============================] - 0s 2ms/step - loss: 9.7557 - accuracy: 0.3947
Epoch 8/50
5/5 [==============================] - 0s 2ms/step - loss: 9.7557 - accuracy: 0.3947
Epoch 9/50
5/5 [==============================] - 0s 2ms/step - loss: 9.7557 - accuracy: 0.3947
Epoch 10/50
5/5 [==============================] - 0s 3ms/step - loss: 9.7557 - accuracy: 0.3947
Epoch 11/50
5/5 [======================

In [24]:
res = model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 1ms/step - loss: 10.7454 - accuracy: 0.3333


In [25]:
res

[10.745396614074707, 0.3333333432674408]

## SVM

In [26]:
X_train

array([[1.02389878e-03, 9.13776945e-04, 6.82989691e-04, ...,
        4.66260544e-04, 9.61842784e-01, 1.43955014e-02],
       [1.87441425e-04, 1.60496720e-04, 4.68603561e-05, ...,
        1.49718838e-03, 9.21023899e-01, 9.91096532e-03],
       [3.30599813e-03, 3.01312090e-03, 1.60848172e-03, ...,
        1.51124649e-03, 9.03202905e-01, 4.46965792e-02],
       ...,
       [5.35262418e-03, 3.85777882e-03, 3.39151828e-03, ...,
        3.84840675e-03, 7.55377226e-01, 8.96989222e-02],
       [5.31865042e-04, 4.18228679e-04, 3.61996251e-04, ...,
        2.09348641e-03, 9.11008669e-01, 1.92396907e-02],
       [3.63636364e-03, 2.69915651e-03, 1.90604499e-03, ...,
        1.96579194e-03, 9.25014058e-01, 4.34594658e-02]])

In [27]:
param_grid = {'C':[2**1], 'gamma':[0.1, 10], 'kernel':['poly']}
gridSVM = GridSearchCV(SVC(), param_grid=param_grid, cv = 5)

#gridSVM = SVC(C=2, gamma=0.1, kernel='poly')
gridSVM.fit(X_train, Y_train)

print('Training Score: ' + str(gridSVM.score(X_train, Y_train)))
print("Testing Score: " + str(gridSVM.score(X_test, Y_test)))

Training Score: 0.625
Testing Score: 0.6818181818181818


In [28]:
gridSVM.best_params_

{'C': 2, 'gamma': 10, 'kernel': 'poly'}

In [29]:
param_grid = {'C':[0.5, 2**1, 2**2, 2**3 ], 'gamma':[0.1, 2, 10, 100], 'kernel':['poly']}
gridSVM = GridSearchCV(SVC(), param_grid=param_grid, cv = 5)

#gridSVM = SVC(C=2, gamma=0.1, kernel='poly')
gridSVM.fit(X_train, Y_train)

print('Training Score: ' + str(gridSVM.score(X_train, Y_train)))
print("Testing Score: " + str(gridSVM.score(X_test, Y_test)))

Training Score: 0.756578947368421
Testing Score: 0.6212121212121212


In [30]:
gridSVM.best_params_

{'C': 8, 'gamma': 100, 'kernel': 'poly'}

In [31]:
param_grid = {'C':[0.5, 2**1, 2**2, 2**3, 2**4, 2**5 ], 'gamma':[0.01, 0.1, 2, 10, 100], 'kernel':['poly']}
gridSVM = GridSearchCV(SVC(), param_grid=param_grid, cv = 5)

#gridSVM = SVC(C=2, gamma=0.1, kernel='poly')
gridSVM.fit(X_train, Y_train)

print('Training Score: ' + str(gridSVM.score(X_train, Y_train)))
print("Testing Score: " + str(gridSVM.score(X_test, Y_test)))

Training Score: 0.756578947368421
Testing Score: 0.6212121212121212


In [32]:
gridSVM.best_params_

{'C': 8, 'gamma': 100, 'kernel': 'poly'}

In [33]:
param_grid = {'C':[0.5, 2**1, 2**2, 2**3, 2**4, 2**5, 2**6, 2**8 ], 'gamma':[0.01, 0.1, 2, 10, 100], 'kernel':['poly', 'linear', 'rbf', 'sigmoid']}
gridSVM = GridSearchCV(SVC(), param_grid=param_grid, cv = 5)

#gridSVM = SVC(C=2, gamma=0.1, kernel='poly')
gridSVM.fit(X_train, Y_train)

print('Training Score: ' + str(gridSVM.score(X_train, Y_train)))
print("Testing Score: " + str(gridSVM.score(X_test, Y_test)))

Training Score: 0.756578947368421
Testing Score: 0.6212121212121212


In [34]:
gridSVM.best_params_

{'C': 8, 'gamma': 100, 'kernel': 'poly'}

In [35]:
param_grid = {'C':[0.5, 2**1, 2**2, 2**3 ], 'gamma':[0.1, 2, 10, 100], 'kernel':['linear']}
gridSVM = GridSearchCV(SVC(), param_grid=param_grid, cv = 5)

#gridSVM = SVC(C=2, gamma=0.1, kernel='poly')
gridSVM.fit(X_train, Y_train)

print('Training Score: ' + str(gridSVM.score(X_train, Y_train)))
print("Testing Score: " + str(gridSVM.score(X_test, Y_test)))

Training Score: 0.6052631578947368
Testing Score: 0.6666666666666666


In [36]:
gridSVM.best_params_

{'C': 0.5, 'gamma': 0.1, 'kernel': 'linear'}

In [37]:
param_grid = {'C':[0.5, 2**1, 2**2, 2**3 ], 'gamma':[0.1, 2, 10, 100], 'kernel':['sigmoid']}
gridSVM = GridSearchCV(SVC(), param_grid=param_grid, cv = 5)

#gridSVM = SVC(C=2, gamma=0.1, kernel='poly')
gridSVM.fit(X_train, Y_train)

print('Training Score: ' + str(gridSVM.score(X_train, Y_train)))
print("Testing Score: " + str(gridSVM.score(X_test, Y_test)))

Training Score: 0.6052631578947368
Testing Score: 0.6666666666666666


In [38]:
gridSVM.best_params_

{'C': 0.5, 'gamma': 0.1, 'kernel': 'sigmoid'}

## KNN

In [39]:
from sklearn.neighbors import KNeighborsClassifier


In [40]:
model = KNeighborsClassifier(n_neighbors=1, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric_params=None, n_jobs=-1)
model.fit(X_train, Y_train)
model.score(X_test, Y_test)

0.6060606060606061

In [41]:
knnAlgos = ['auto', 'ball_tree', 'kd_tree', 'brute']

model = KNeighborsClassifier(n_neighbors=1, weights='uniform', algorithm=knnAlgos[3], leaf_size=30, p=2, metric_params=None, n_jobs=-1)
model.fit(X_train, Y_train)
model.score(X_test, Y_test)

0.6060606060606061

In [42]:
model = KNeighborsClassifier(n_neighbors=3, weights='uniform', algorithm='ball_tree', leaf_size=30, p=2, metric_params=None, n_jobs=-1)
model.fit(X_train, Y_train)
model.score(X_test, Y_test)

0.6363636363636364

In [43]:
model = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric_params=None, n_jobs=-1)
model.fit(X_train, Y_train)
model.score(X_test, Y_test)

0.5454545454545454

In [44]:
model = KNeighborsClassifier(n_neighbors=7, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric_params=None, n_jobs=-1)
model.fit(X_train, Y_train)
model.score(X_test, Y_test)

0.6212121212121212

In [45]:
model = KNeighborsClassifier(n_neighbors=21, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric_params=None, n_jobs=-1)
model.fit(X_train, Y_train)
model.score(X_test, Y_test)

0.6515151515151515

In [46]:
model = KNeighborsClassifier(n_neighbors=21, weights='uniform', algorithm='kd_tree', leaf_size=30, p=2, metric_params=None, n_jobs=-1)
model.fit(X_train, Y_train)
model.score(X_test, Y_test)

0.6515151515151515

In [47]:
model = KNeighborsClassifier(n_neighbors=61, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric_params=None, n_jobs=-1)
model.fit(X_train, Y_train)
model.score(X_test, Y_test)

0.6666666666666666

In [48]:
# Grid Search
param_grid = {'n_neighbors':[1, 3, 5, 21, 61], 'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}

gridKNN = GridSearchCV(KNeighborsClassifier(), param_grid=param_grid, cv=5)
gridKNN.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'n_neighbors': [1, 3, 5, 21, 61]})

In [49]:
gridKNN.score(X_test, Y_test)

0.5454545454545454

In [50]:
gridKNN.best_score_

0.6503225806451612

In [51]:
gridKNN.best_params_

{'algorithm': 'auto', 'n_neighbors': 5}

# Random Forest

In [52]:
param_grid = {'n_estimators':[1500, 1800, 2000]}
#gridRFT = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv = 5)
gridRFT = RandomForestClassifier(n_estimators=10)
gridRFT.fit(X_train, Y_train)

print('Training Score: ' + str(gridRFT.score(X_train, Y_train)))
print("Testing Score: " + str(gridRFT.score(X_test, Y_test)))

Training Score: 0.9868421052631579
Testing Score: 0.6060606060606061


In [53]:
param_grid = {'n_estimators':[1500, 1800, 2000]}
#gridRFT = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv = 5)
gridRFT = RandomForestClassifier(n_estimators=20)
gridRFT.fit(X_train, Y_train)

print('Training Score: ' + str(gridRFT.score(X_train, Y_train)))
print("Testing Score: " + str(gridRFT.score(X_test, Y_test)))

Training Score: 0.993421052631579
Testing Score: 0.5606060606060606


In [54]:
param_grid = {'n_estimators':[1500, 1800, 2000]}
#gridRFT = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv = 5)
gridRFT = RandomForestClassifier(n_estimators=30)
gridRFT.fit(X_train, Y_train)

print('Training Score: ' + str(gridRFT.score(X_train, Y_train)))
print("Testing Score: " + str(gridRFT.score(X_test, Y_test)))

Training Score: 1.0
Testing Score: 0.6818181818181818


In [55]:
param_grid = {'n_estimators':[1500, 1800, 2000]}
#gridRFT = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv = 5)
gridRFT = RandomForestClassifier(n_estimators=50)
gridRFT.fit(X_train, Y_train)

print('Training Score: ' + str(gridRFT.score(X_train, Y_train)))
print("Testing Score: " + str(gridRFT.score(X_test, Y_test)))

Training Score: 1.0
Testing Score: 0.5454545454545454


In [56]:
param_grid = {'n_estimators':[1500, 1800, 2000]}
#gridRFT = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv = 5)
gridRFT = RandomForestClassifier(n_estimators=500)
gridRFT.fit(X_train, Y_train)

print('Training Score: ' + str(gridRFT.score(X_train, Y_train)))
print("Testing Score: " + str(gridRFT.score(X_test, Y_test)))

Training Score: 1.0
Testing Score: 0.6212121212121212


In [57]:
param_grid = {'n_estimators':[1500, 1800, 2000]}
#gridRFT = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv = 5)
gridRFT = RandomForestClassifier(n_estimators=700)
gridRFT.fit(X_train, Y_train)

print('Training Score: ' + str(gridRFT.score(X_train, Y_train)))
print("Testing Score: " + str(gridRFT.score(X_test, Y_test)))

Training Score: 1.0
Testing Score: 0.6060606060606061


In [58]:
param_grid = {'n_estimators':[1500, 1800, 2000]}
#gridRFT = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv = 5)
gridRFT = RandomForestClassifier(n_estimators=1000)
gridRFT.fit(X_train, Y_train)

print('Training Score: ' + str(gridRFT.score(X_train, Y_train)))
print("Testing Score: " + str(gridRFT.score(X_test, Y_test)))

Training Score: 1.0
Testing Score: 0.6212121212121212


In [59]:
param_grid = {'n_estimators':[1500, 1800, 2000]}
#gridRFT = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv = 5)
gridRFT = RandomForestClassifier(n_estimators=1200)
gridRFT.fit(X_train, Y_train)

print('Training Score: ' + str(gridRFT.score(X_train, Y_train)))
print("Testing Score: " + str(gridRFT.score(X_test, Y_test)))

Training Score: 1.0
Testing Score: 0.6212121212121212


In [60]:
param_grid = {'n_estimators':[1500, 1800, 2000]}
#gridRFT = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv = 5)
gridRFT = RandomForestClassifier(n_estimators=3500)
gridRFT.fit(X_train, Y_train)

print('Training Score: ' + str(gridRFT.score(X_train, Y_train)))
print("Testing Score: " + str(gridRFT.score(X_test, Y_test)))

Training Score: 1.0
Testing Score: 0.5757575757575758


# MLP

In [61]:
from sklearn.neural_network import MLPClassifier


In [62]:

grid_params = {'hidden_layer_sizes':[(50,), (100), (50, 100,)], 'activation':['identity', 'logistic', 'tanh', 'relu'], 'solver':['lbfgs', 'sgd', 'adam'] }
gridMLP = GridSearchCV(MLPClassifier(), param_grid=grid_params, cv=5)
gridMLP.fit(X_train, Y_train)

C:\Users\H P\Envs\mldsenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\H P\Envs\mldsenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\H P\Envs\mldsenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of 

C:\Users\H P\Envs\mldsenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\H P\Envs\mldsenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\H P\Envs\mldsenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of 

GridSearchCV(cv=5, estimator=MLPClassifier(),
             param_grid={'activation': ['identity', 'logistic', 'tanh', 'relu'],
                         'hidden_layer_sizes': [(50,), 100, (50, 100)],
                         'solver': ['lbfgs', 'sgd', 'adam']})

In [63]:
gridMLP.best_score_

0.6384946236559139

In [64]:
gridMLP.best_params_

{'activation': 'logistic', 'hidden_layer_sizes': (50,), 'solver': 'lbfgs'}

In [65]:
mlpCls = MLPClassifier(hidden_layer_sizes=(100,), activation='identity', solver='adam' )
mlpCls.fit(X_train, Y_train)
mlpCls.score(X_test, Y_test)

0.6666666666666666

In [66]:
# grid_params = {'hidden_layer_sizes':[(50,), (100), (50, 100,)], 'activation':['identity', 'logistic', 'tanh', 'relu'], 'solver':['lbfgs', 'sgd', 'adam'] }
mlpCls = MLPClassifier(hidden_layer_sizes=(100,), activation='logistic', solver='adam' )
mlpCls.fit(X_train, Y_train)
mlpCls.score(X_test, Y_test)

0.6666666666666666

In [67]:
# grid_params = {'hidden_layer_sizes':[(50,), (100), (50, 100,)], 'activation':['identity', 'logistic', 'tanh', 'relu'], 'solver':['lbfgs', 'sgd', 'adam'] }
mlpCls = MLPClassifier(hidden_layer_sizes=(100,), activation='logistic', solver='sgd' )
mlpCls.fit(X_train, Y_train)
mlpCls.score(X_test, Y_test)

0.6666666666666666

In [68]:
# grid_params = {'hidden_layer_sizes':[(50,), (100), (50, 100,)], 'activation':['identity', 'logistic', 'tanh', 'relu'], 'solver':['lbfgs', 'sgd', 'adam'] }
mlpCls = MLPClassifier(hidden_layer_sizes=(100,), activation='logistic', solver='lbfgs' )
mlpCls.fit(X_train, Y_train)
mlpCls.score(X_test, Y_test)

0.6666666666666666

In [69]:
# grid_params = {'hidden_layer_sizes':[(50,), (100), (50, 100,)], 'activation':['identity', 'logistic', 'tanh', 'relu'], 'solver':['lbfgs', 'sgd', 'adam'] }
mlpCls = MLPClassifier(hidden_layer_sizes=(50,), activation='identity', solver='adam' )
mlpCls.fit(X_train, Y_train)
mlpCls.score(X_test, Y_test)

0.6666666666666666

In [70]:
# grid_params = {'hidden_layer_sizes':[(50,), (100), (50, 100,)], 'activation':['identity', 'logistic', 'tanh', 'relu'], 'solver':['lbfgs', 'sgd', 'adam'] }
mlpCls = MLPClassifier(hidden_layer_sizes=(200,), activation='identity', solver='adam' )
mlpCls.fit(X_train, Y_train)
mlpCls.score(X_test, Y_test)

0.6666666666666666

In [71]:
# grid_params = {'hidden_layer_sizes':[(50,), (100), (50, 100,)], 'activation':['identity', 'logistic', 'tanh', 'relu'], 'solver':['lbfgs', 'sgd', 'adam'] }
mlpCls = MLPClassifier(hidden_layer_sizes=(50, 100,), activation='identity', solver='adam' )
mlpCls.fit(X_train, Y_train)
mlpCls.score(X_test, Y_test)

0.6666666666666666